In [40]:
import numpy as np
import csv
from sklearn.cluster import KMeans
import pandas as pd
import pickle
from sklearn import preprocessing
train_data = pd.read_csv('train.csv')

#train = pd.read_csv('train.csv', iterator=True)
# train_data = train.get_chunk(size=10)
profiles_file = "profiles.csv"
ids = []
profiles_data = {}
with open(profiles_file, 'r') as profiles_fh:
    profiles_csv = csv.reader(profiles_fh, delimiter=',', quotechar='"')
    next(profiles_csv, None)
    for row in profiles_csv:
        user = row[0]
        sex = row[1]
        age  = row[2]
        if age == '':
            age = 0
        country = row[3]
        
        if not user in profiles_data:
            profiles_data[user] = {}
            ids.append(user)
        
        profiles_data[user]['sex'] = sex
        profiles_data[user]['age'] = int(age)
        profiles_data[user]['country'] = country

# go through every line in training data. match for the user
# if artistplays column exists already for the given artist, add to that value
# if not instantiate it
        
feature_columns = ['Male', 'Female', 'Num Artists', 'Num Plays']
# feature_columns = ['Male', 'Female']
for user in ids:
    profiles_data[user]
    if profiles_data[user]['age'] not in feature_columns:
        feature_columns.append(profiles_data[user]['age'])
    if profiles_data[user]['country'] not in feature_columns:
        feature_columns.append(profiles_data[user]['country'])

feature_matrix = np.zeros((len(ids), len(feature_columns)))
user_num_dict = {}
for n, user in enumerate(ids):
    user_num_dict[user] = n
    if profiles_data[user]['sex'] == 'm':
        feature_matrix[n,0] = 1
    elif profiles_data[user]['sex'] == 'f':
        feature_matrix[n,1] = 1
    feature_matrix[n, feature_columns.index(profiles_data[user]['country'])] = 1
    feature_matrix[n, feature_columns.index(profiles_data[user]['age'])] = 1

    num_artists = 0
    num_plays = 0

for _, row in train_data.iterrows():  
    if row['user'] in user_num_dict:
        feature_matrix[user_num_dict[row['user']], 2] += 1
        feature_matrix[user_num_dict[row['user']], 3] += row['plays']

# sums = feature_matrix.sum(axis=0)
# artists_sum = sums[2]
# plays_sum = sums[3]

# # normalize
# for row in feature_matrix:
#     row[2] = row[2] / artists_sum
#     row[3] = row[3] / plays_sum

# train_file = 'train.csv'
# test_file  = 'test.csv'
# soln_file  = 'user_median.csv'
print(feature_matrix.shape)

normalized_features = preprocessing.normalize(feature_matrix, axis=0)
print(normalized_features.shape)
# np.savetxt('file_2.csv', normalized_features, delimiter=",")

(233286, 357)
(233286, 357)


In [42]:

#profiles = "profiles.csv"
#data = pd.read_csv(profiles)
num_clusters = 40
km = KMeans(n_clusters = num_clusters, init='k-means++', max_iter=1000, n_init=5)

preds = np.asarray(km.fit_predict(normalized_features))
print km.score(normalized_features)
new_preds = np.transpose(preds)
new_preds2 = new_preds.reshape((new_preds.shape[0], 1))

clustered_features = np.append(normalized_features, new_preds2, axis=1)
pickle.dump( clustered_features, open( "clustered_features.p", "wb" ) )
#np.savetxt('clustered_features.csv', final_features, delimiter=",")

-314.121774025
